In [1]:
import pandas as pd
import sys
import json
import os
import re
# split the data into train and validation
from sklearn.model_selection import train_test_split
import warnings
from collections import Counter
warnings.filterwarnings("ignore")

In [2]:
bert_path = "../../DeepHateLingo/training-script/bangla-bert-base/output/"
xlm_path = "../../DeepHateLingo/training-script/xlm-roberta/output/"
bangla_bert = pd.read_csv(f"{bert_path}six_bangla_bert_last_two_pred_3e5.csv")
mbert_cased = pd.read_csv(f"{bert_path}six_bert_base_multilingual_cased_last_four_pred_2e5.csv")
mbert_uncased = pd.read_csv(f"{bert_path}six_bert_base_multilingual_uncased_last_two_pred_5e5.csv")
roberta = pd.read_csv(f"{xlm_path}five_xlm_roberta_pool_pred_2e5.csv")

In [3]:
target = []
bangla_bert_pred  = []
mbert_cased_pred = []
mbert_uncased_pred = []
roberta_pred = []
for _id in range(len(bangla_bert)):
    target.append(bangla_bert['target'][_id])
    bangla_bert_pred.append(bangla_bert['y_pred'][_id])
    mbert_cased_pred.append(mbert_cased['y_pred'][_id])
    mbert_uncased_pred.append(mbert_uncased['y_pred'][_id])
    roberta_pred.append(roberta['y_pred'][_id])

In [4]:
mx_vote_df = pd.DataFrame()
mx_vote_df['target'] = target
mx_vote_df['bangla-bert'] = bangla_bert_pred
mx_vote_df['mbert-cased'] = mbert_cased_pred
mx_vote_df['mbert-uncased'] = mbert_uncased_pred
mx_vote_df['roberta'] = roberta_pred

In [5]:
mx_vote_df[:10]

,target,bangla-bert,mbert-cased,mbert-uncased,roberta
0,2,2,0,0,2
1,0,0,0,0,0
2,2,2,2,2,2
3,0,0,0,0,0
4,1,0,0,0,0
5,2,2,2,2,2
6,3,3,3,3,3
7,0,0,0,0,0
8,1,1,1,1,1
9,0,0,0,0,0


In [6]:
print("bangla_bert = \n", bangla_bert.y_pred.value_counts())
print("mbert_cased = \n", mbert_cased.y_pred.value_counts())
print("mbert_uncased = \n", mbert_uncased.y_pred.value_counts())
print("roberta = \n", roberta.y_pred.value_counts())

bangla_bert = 
 0    519
3    360
2    176
1    145
Name: y_pred, dtype: int64
mbert_cased = 
 0    550
3    351
2    162
1    137
Name: y_pred, dtype: int64
mbert_uncased = 
 0    521
3    348
2    178
1    153
Name: y_pred, dtype: int64
roberta = 
 0    507
3    364
2    186
1    143
Name: y_pred, dtype: int64


In [7]:
print("voting bangla-bert = \n", mx_vote_df['bangla-bert'].value_counts())
print("voting mbert-cased = \n", mx_vote_df['mbert-cased'].value_counts())
print("voting mbert-uncased = \n", mx_vote_df['mbert-uncased'].value_counts())
print("voting roberta = \n",mx_vote_df['roberta'].value_counts()) 	 	 	

voting bangla-bert = 
 0    519
3    360
2    176
1    145
Name: bangla-bert, dtype: int64
voting mbert-cased = 
 0    550
3    351
2    162
1    137
Name: mbert-cased, dtype: int64
voting mbert-uncased = 
 0    521
3    348
2    178
1    153
Name: mbert-uncased, dtype: int64
voting roberta = 
 0    507
3    364
2    186
1    143
Name: roberta, dtype: int64


In [8]:
mx_vote_df = mx_vote_df[['target','bangla-bert','mbert-uncased','roberta']]

In [9]:
preds = []
for index in range(len(mx_vote_df)):
    values = mx_vote_df.iloc[index].values[1:]
    sorted_x = sorted(Counter(values).items(), key=lambda kv: kv[1],reverse=True)
    preds.append(sorted_x[0][0])
##preds.append(math.floor(sum(values)/5))
mx_vote_df['preds'] = preds

In [10]:
mx_vote_df

,target,bangla-bert,mbert-uncased,roberta,preds
0,2,2,0,2,2
1,0,0,0,0,0
2,2,2,2,2,2
3,0,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
1195,0,0,0,0,0
1196,0,0,2,2,2
1197,2,2,2,2,2
1198,0,0,0,0,0


In [11]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef
y_test = mx_vote_df.target.values
y_pred = mx_vote_df['preds'].values
print('Accuracy::', metrics.accuracy_score(y_test, y_pred))
print('Mcc Score::', matthews_corrcoef(y_test, y_pred))
print('Precision::', metrics.precision_score(y_test, y_pred, average='weighted'))
print('Recall::', metrics.recall_score(y_test, y_pred, average='weighted'))
print('F_score::', metrics.f1_score(y_test, y_pred, average='weighted'))
print('classification_report:: \n', metrics.classification_report(y_test, y_pred))

Accuracy:: 0.8766666666666667
Mcc Score:: 0.8205216999232595
Precision:: 0.8775601042666743
Recall:: 0.8766666666666667
F_score:: 0.8763746343903924
classification_report:: 
               precision    recall  f1-score   support

           0       0.91      0.90      0.91       524
           1       0.82      0.74      0.78       157
           2       0.79      0.90      0.84       159
           3       0.89      0.89      0.89       360

    accuracy                           0.88      1200
   macro avg       0.85      0.86      0.85      1200
weighted avg       0.88      0.88      0.88      1200

